# AtomsBase integration

[AtomsBase.jl](https://github.com/JuliaMolSim/AtomsBase.jl) is a common interface
for representing atomic structures in Julia. DFTK directly supports using such
structures to run a calculation as is demonstrated here.

In [1]:
using DFTK
using AtomsBuilder

## Feeding an AtomsBase AbstractSystem to DFTK

In this example we construct a bulk silicon system using the `bulk` function
from [AtomsBuilder](https://github.com/JuliaMolSim/AtomsBuilder.jl). This function
uses tabulated data to set up a reasonable starting geometry and lattice for bulk silicon.

In [2]:
system = bulk(:Si)

FlexibleSystem(Si₂, periodicity = TTT):
    cell_vectors      : [       0    2.715    2.715;
                            2.715        0    2.715;
                            2.715    2.715        0]u"Å"

    Atom(Si, [       0,        0,        0]u"Å")
    Atom(Si, [  1.3575,   1.3575,   1.3575]u"Å")


By default the atoms of an `AbstractSystem` employ the bare Coulomb potential.
To employ pseudpotential models (which is almost always advisable for
plane-wave DFT) one employs the `pseudopotential` keyword argument in
model constructors such as `model_DFT`.
For example we can employ a `PseudoFamily` object
from the [PseudoPotentialData](https://github.com/JuliaMolSim/PseudoPotentialData.jl)
package. See its documentation for more information on the available
pseudopotential families and how to select them.

In [3]:
using PseudoPotentialData  # defines PseudoFamily

pd_lda_family = PseudoFamily("dojo.nc.sr.lda.v0_4_1.standard.upf")
model = model_DFT(system; functionals=LDA(), temperature=1e-3,
                  pseudopotentials=pd_lda_family)

Model(lda_x+lda_c_pw, 3D):
    lattice (in Bohr)    : [0         , 5.13061   , 5.13061   ]
                           [5.13061   , 0         , 5.13061   ]
                           [5.13061   , 5.13061   , 0         ]
    unit cell volume     : 270.11 Bohr³

    atoms                : Si₂
    pseudopot. family    : PseudoFamily("dojo.nc.sr.lda.v0_4_1.standard.upf")

    num. electrons       : 8
    spin polarization    : none
    temperature          : 0.001 Ha
    smearing             : DFTK.Smearing.FermiDirac()

    terms                : Kinetic()
                           AtomicLocal()
                           AtomicNonlocal()
                           Ewald(nothing)
                           PspCorrection()
                           Hartree()
                           Xc(lda_x, lda_c_pw)
                           Entropy()

Alternatively the `pseudopotentials` object also accepts a `Dict{Symbol,String}`,
which provides for each element symbol the filename or identifier
of the pseudopotential to be employed, e.g.

In [4]:
path_to_pspfile = PseudoFamily("cp2k.nc.sr.lda.v0_1.semicore.gth")[:Si]
model = model_DFT(system; functionals=LDA(), temperature=1e-3,
                  pseudopotentials=Dict(:Si => path_to_pspfile))

Model(lda_x+lda_c_pw, 3D):
    lattice (in Bohr)    : [0         , 5.13061   , 5.13061   ]
                           [5.13061   , 0         , 5.13061   ]
                           [5.13061   , 5.13061   , 0         ]
    unit cell volume     : 270.11 Bohr³

    atoms                : Si₂
    atom potentials      : ElementPsp(:Si, "/home/runner/.julia/artifacts/966fd9cdcd7dbaba6dc2bf43ee50dd81e63e8837/Si.gth")
                           ElementPsp(:Si, "/home/runner/.julia/artifacts/966fd9cdcd7dbaba6dc2bf43ee50dd81e63e8837/Si.gth")

    num. electrons       : 8
    spin polarization    : none
    temperature          : 0.001 Ha
    smearing             : DFTK.Smearing.FermiDirac()

    terms                : Kinetic()
                           AtomicLocal()
                           AtomicNonlocal()
                           Ewald(nothing)
                           PspCorrection()
                           Hartree()
                           Xc(lda_x, lda_c_pw)
                 

We can then discretise such a model and solve:

In [5]:
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921728150136                   -0.69    5.6    231ms
  2   -7.926131087284       -2.36       -1.22    1.0    146ms
  3   -7.926834040387       -3.15       -2.37    2.0    249ms
  4   -7.926861305968       -4.56       -3.02    3.1    209ms
  5   -7.926861656280       -6.46       -3.47    2.0    181ms
  6   -7.926861675599       -7.71       -4.05    1.6    160ms
  7   -7.926861678225       -8.58       -4.08    2.1    178ms
  8   -7.926861679009       -9.11       -4.14    1.0    151ms
  9   -7.926861681730       -8.57       -4.37    1.0    150ms
 10   -7.926861681844       -9.94       -4.53    1.0    149ms
 11   -7.926861681853      -11.05       -4.58    1.0    152ms
 12   -7.926861681867      -10.86       -5.05    1.0    231ms
 13   -7.926861681871      -11.35       -5.35    1.1    866ms
 14   -7.926861681872      -12.32       -5.22    1.0    147ms
 15   -7.

If we did not want to use
[AtomsBuilder](https://github.com/JuliaMolSim/AtomsBuilder.jl)
we could of course use any other package
which yields an AbstractSystem object. This includes:

### Reading a system using AtomsIO

Read a file using [AtomsIO](https://github.com/mfherbst/AtomsIO.jl),
which directly yields an AbstractSystem.

In [6]:
using AtomsIO
system = load_system("Si.extxyz");

Run the LDA calculation:

In [7]:
pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.semicore.gth")
model  = model_DFT(system; pseudopotentials, functionals=LDA(), temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921747015495                   -0.69    5.5    210ms
  2   -7.926130570805       -2.36       -1.22    1.0    230ms
  3   -7.926834130054       -3.15       -2.37    2.0    167ms
  4   -7.926861048379       -4.57       -3.02    3.1    240ms
  5   -7.926861665677       -6.21       -3.47    2.1    173ms
  6   -7.926861676819       -7.95       -4.09    1.5    159ms
  7   -7.926861681626       -8.32       -4.77    1.9    162ms
  8   -7.926861681863       -9.63       -5.54    2.6    189ms
  9   -7.926861681872      -11.08       -5.64    2.2    179ms
 10   -7.926861681872      -12.10       -6.30    1.0    150ms
 11   -7.926861681873      -12.84       -7.49    1.6    162ms
 12   -7.926861681873      -14.45       -7.93    3.5    211ms
 13   -7.926861681873   +  -14.75       -8.72    3.0    187ms


The same could be achieved using [ExtXYZ](https://github.com/libAtoms/ExtXYZ.jl)
by `system = Atoms(read_frame("Si.extxyz"))`,
since the `ExtXYZ.Atoms` object is directly AtomsBase-compatible.

### Directly setting up a system in AtomsBase

In [8]:
using AtomsBase
using Unitful
using UnitfulAtomic

# Construct a system in the AtomsBase world
a = 10.26u"bohr"  # Silicon lattice constant
lattice = a / 2 * [[0, 1, 1.],  # Lattice as vector of vectors
                   [1, 0, 1.],
                   [1, 1, 0.]]
atoms  = [:Si => ones(3)/8, :Si => -ones(3)/8]
system = periodic_system(atoms, lattice; fractional=true)

# Now run the LDA calculation:
pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.semicore.gth")
model  = model_DFT(system; pseudopotentials, functionals=LDA(), temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921700723895                   -0.69    5.6    250ms
  2   -7.926135986992       -2.35       -1.22    1.0    127ms
  3   -7.926836667531       -3.15       -2.37    2.0    173ms
  4   -7.926864667646       -4.55       -2.99    3.0    198ms
  5   -7.926865056768       -6.41       -3.38    1.8    152ms
  6   -7.926865081718       -7.60       -3.84    1.6    141ms
  7   -7.926865090611       -8.05       -4.16    1.8    148ms


## Obtaining an AbstractSystem from DFTK data

At any point we can also get back the DFTK model as an
AtomsBase-compatible `AbstractSystem`:

In [9]:
second_system = atomic_system(model)

FlexibleSystem(Si₂, periodicity = TTT):
    cell_vectors      : [       0     5.13     5.13;
                             5.13        0     5.13;
                             5.13     5.13        0]u"a₀"

    Atom(Si, [  1.2825,   1.2825,   1.2825]u"a₀")
    Atom(Si, [ -1.2825,  -1.2825,  -1.2825]u"a₀")


Similarly DFTK offers a method to the `atomic_system` and `periodic_system` functions
(from AtomsBase), which enable a seamless conversion of the usual data structures for
setting up DFTK calculations into an `AbstractSystem`:

In [10]:
lattice = 5.431u"Å" / 2 * [[0 1 1.];
                           [1 0 1.];
                           [1 1 0.]];
Si = ElementPsp(:Si, pseudopotentials)
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

third_system = atomic_system(lattice, atoms, positions)

FlexibleSystem(Si₂, periodicity = TTT):
    cell_vectors      : [       0  5.13155  5.13155;
                          5.13155        0  5.13155;
                          5.13155  5.13155        0]u"a₀"

    Atom(Si, [ 1.28289,  1.28289,  1.28289]u"a₀")
    Atom(Si, [-1.28289, -1.28289, -1.28289]u"a₀")
